# How often does the European Union express "deep concern"?

During the climax of the war in Ukraine, in 2014-2015, many Ukrainian people felt that the European Union failed to provide them with real support, contenting themselves with releasing statements that barely condemned Russia and mostly expressed "deep concern" about the "situation" in Ukraine. These catch phrases have become a source of sarcastic jokes, but also an illustration, beyond the Ukrainian case, of the EU's toothless "declaratory diplomacy".

In order to check whether the EU does indeed express "concern" or "deep concern" about every "situation" in the world, we will read through the EU foreign policy statements and identify the most mentionned countries as well as the most frequent words used to express the EU's position.

The source of the data is the [website of the Council of the European Union](https://www.consilium.europa.eu/en/press/press-releases/?keyword=&dateFrom=&dateTo=2020%2F06%2F18&filters=1653&filters=1470&filters=1474), where all press releases and statements since 2014 are online, though the oldest only in PDF format. We will scrap those published before June 18th, 2020.

In [ ]:
# Importing libraries needed for web scraping
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from datetime import datetime

Before we scrap full texts of press releases and statements, we first scrap only titles, dates and links from search results so that we will able to select only the content relevant for our analysis.

## Step 1: building a table of contents to facilitate content selection

In [ ]:
data = []

# There are 115 pages of search results
for i in tqdm(range(1, 116)):
    page_nb = str(i)
    url = 'https://www.consilium.europa.eu/en/press/press-releases/?dateTo=2020%2f06%2f18&filters=1653&filters=1470&filters=1474&Page=' + page_nb
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    rows = soup.select('div .inline-items')
    
    for row in rows:
        d = dict()
        raw_date = row.span.attrs['content']
        d['date'] = datetime.fromisoformat(raw_date)
        d['link'] = 'https://www.consilium.europa.eu' + row.select_one('a')['href']
        d['title'] = row.select_one('a').text
        data.append(d)
    
    sleep(10)

In [ ]:
# Generating a dataframe
import pandas as pd
df = pd.DataFrame(data=data)
df.head()

In [ ]:
# Saving the data to a JSON file
df.to_json('data.json', orient='table')

In [ ]:
# New starting point
import pandas as pd
df = pd.read_json('data.json', orient='table')

In [ ]:
# Filtering content
df = df.loc[df['title'].str.contains('Declaration on|High Representative|Statement of the EU Foreign Ministers|Foreign Affairs Council|conclusions', case=False)]
# We focus here on positions towards states and diplomatic crises,
# leaving aside issue-centered positions on e.g. climate, development and human rights,
# or statements related to processes like enlargement and bilateral cooperation
df = df.loc[(df['title'].str.contains('indicative|climate|water|migration|gender|eea|day|child|indigenous|anniversary|strategy|development policy|eastern partnership|alignment', case=False)) == False]
df = df.drop([2192, 2144, 2121, 2116, 2110, 2108, 2107, 2100, 2089, 2068, 2078, 2073, 2069, 2042, 2016, 2011, 2010, 2009, 1992, 1991, 1990, 1989, 1972, 1970, 1964, 1960, 1958, 1954, 1896, 1890, 1882, 1881, 1853, 1849, 1829, 1828, 1809, 1806, 1796, 1794, 1793, 1792, 1789, 1784, 1782, 1781, 1779, 1717, 1704, 1699, 1678, 1677, 1676, 1647, 1638, 1636, 1634, 1603, 1576, 1541, 1500, 1499, 1460, 1454, 1426, 1423, 1422, 1414, 1393, 1391, 1378, 1354, 1344, 1322, 1321, 1317, 1300, 1299, 1298, 1296, 1295, 1294, 1286, 1284, 1283, 1263, 1261, 1259, 1248, 1229, 1198, 1159, 1131, 1126, 1124, 1114, 1112, 1103, 1102, 1087, 1042, 1027, 1022, 982, 978, 976, 975, 974, 946, 945, 941, 940, 893, 832, 825, 818, 784, 781, 745, 703, 702, 645, 644, 614, 606, 599, 537, 534, 508, 484, 483, 469, 467, 442, 417, 379, 365, 363, 361, 357, 333, 331, 328, 294, 293, 292, 284, 264, 228, 204, 196, 175, 174, 158, 157, 155, 154, 134, 105, 99, 64, 47, 28, 9, 8, 4, 2])

In [ ]:
# Importing a list of countries from Google Public Data
list_countries = pd.read_csv('https://raw.githubusercontent.com/google/dspl/master/samples/google/canonical/countries.csv', usecols=['name'])
list_countries = pd.Series(data=list_countries['name']).tolist()

In [ ]:
# Custom function to look for country names in titles 
def find_country(sentence):
    split_sentence = sentence.split()
    for word in split_sentence:
        word = word.strip(' :').replace("'s", "")
        if word in list_countries:
            return word
            break

df['main_topic'] = df['title'].apply(find_country)

In [ ]:
custom_dict = {
    "Democratic Republic of the Congo": "Congo [DRC]",
    "Democratic Republic of Congo": "Congo [DRC]",
    "DRC": "Congo [DRC]",
    "Republic of the Congo": "Congo [Republic]",
    "Central African Republic": "Central African Republic",
    "République centrafricaine": "Central African Republic",
    "Côte d'Ivoire": "Côte d'Ivoire",
    "Tripoli": "Libya",
    "Aleppo": "Syria",
    "Middle East": "Middle East",
    "Gaza": "Middle East",
    "Palestinian Authority": "Middle East",
    "United States": "United States",
    "Hong Kong": "China",
    "Myanmar/Burma": "Myanmar [Burma]",
    "Sri Lanka": "Sri Lanka",
    "Democratic People's Republic of Korea": "North Korea",
    "Bosnia and Herzegovina": "Bosnia and Herzegovina",
    "Crimea": "Ukraine",
    "MH17": "Ukraine",
    "UKRAINE": "Ukraine",
    "Azov Sea": "Ukraine",
    "Eastern Mediterranean": "Turkey",
    "Swiss Confederation": "Switzerland",
    "ISIL": "terrorism",
    "terrorism": "terrorism",
    "Boko Haram": "terrorism",
    "Ebola": "ebola",
    "Khashoggi": "Saudi Arabia"
}

def find_topic(sentence):
    for item in custom_dict:
        if item in sentence:
            return custom_dict[item]
            break
        
df.loc[df['main_topic'].isnull(), 'main_topic'] = df['title'].apply(find_topic)

In [ ]:
df.loc[df['title'].str.contains('South Sudan'), 'main_topic'] = 'South Sudan'
df.loc[df['title'].str.contains('Al-Bashir'), 'main_topic'] = 'Sudan'
df.loc[2087, 'main_topic'] = 'Uganda'
df.loc[1940, 'main_topic'] = 'Ukraine'
df.loc[1937, 'main_topic'] = 'Middle East'
df.loc[1924, 'main_topic'] = 'Ukraine'
df.loc[1914, 'main_topic'] = 'Congo [DRC]'
df.loc[1880, 'main_topic'] = 'Ukraine'
df.loc[1832, 'main_topic'] = 'terrorism'
df.loc[1823, 'main_topic'] = 'ebola'
df.loc[1771, 'main_topic'] = 'Ukraine'
df.loc[1707, 'main_topic'] = 'Ukraine'
df.loc[1674, 'main_topic'] = 'Macedonia [FYROM]'
df.loc[1522, 'main_topic'] = 'Iraq'
df.loc[1507, 'main_topic'] = 'Syria'
df.loc[1258, 'main_topic'] = 'Mali'
df.loc[1161, 'main_topic'] = 'Syria'
df.loc[1083, 'main_topic'] = 'Ukraine'
df.loc[1080, 'main_topic'] = 'Syria'
df.loc[894, 'main_topic'] = 'North Korea'
df.loc[729, 'main_topic'] = 'Russia'
df.loc[513, 'main_topic'] = 'Ukraine'
df.loc[463, 'main_topic'] = 'Yemen'
df.loc[447, 'main_topic'] = 'Venezuela'
df.loc[374, 'main_topic'] = 'Middle East'
df.loc[255, 'main_topic'] = 'Russia'
df.loc[89, 'main_topic'] = 'Turkey'
df.loc[61, 'main_topic'] = 'Syria'
df.loc[52, 'main_topic'] = 'coronavirus'

In [ ]:
print('Remaining items to classify:', df[df['main_topic'].isnull()].shape[0])
print('Final number of documents to analyse:', df.shape[0])

## Step 2: HTML content scraping

In [ ]:
# Sorting documents in HTML and PDF format
df['doctype'] = 'html'
df.loc[df['link'].str.contains('pdf'), 'doctype'] = 'pdf'

In [ ]:
# Importing libraries needed for web scraping
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

In [ ]:
tqdm.pandas()

def get_content(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        text = soup.select('div .col-md-9')[1].select('p')
        content = ''
        for paragraph in text:
            content = content + paragraph.get_text() + ' '
        return content.strip()
    except TypeError:
        pass

df.loc[df['doctype'] == 'html', 'content'] = df.loc[df['doctype'] == 'html', 'link'].progress_apply(get_content)

In [ ]:
df['content'] = df['content'].str.replace('\xa0', '')
df = df.drop([1021, 1056, 1231, 1507])

In [ ]:
ol_items = [200, 201, 203, 205, 769, 834, 984, 1167, 1172, 1213, 1220, 1256, 1319, 1355, 1386, 1387, 1389, 1390, 1421, 1513, 1606, 1721, 1740, 1742, 1743, 1744, 1745, 1746, 1747]
for item in tqdm(ol_items):
    url = df.loc[item, 'link']
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    text = soup.select('div .col-md-9')[1].find('ol').text.replace('\n', ' ').replace('\xa0', '').strip()
    df.loc[item, 'content'] = text
    sleep(10)

In [ ]:
pdf_links = [700, 786, 1069, 1080, 1083, 1089, 1168, 1171, 1218, 1219, 1258, 1288, 1289, 1515, 1516]
for item in tqdm(pdf_links):
    df.loc[item, 'doctype'] = 'pdf'
    url = df.loc[item, 'link']
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    df.loc[item, 'link'] = soup.select_one('.link-pdf')['href']
    sleep(10)

In [ ]:
# Fixing broken links
df.loc[1008, 'doctype'] = 'pdf'
df.loc[1008, 'link'] = 'https://data.consilium.europa.eu/doc/document/ST-7614-2017-INIT/en/pdf'
df.loc[(df['doctype'] == 'pdf') & (df['link'].str.match('http') == False), 'link'] = 'https://www.consilium.europa.eu' + df['link']

## Step 3: PDF content scraping

In [ ]:
import urllib
from io import BytesIO
import PyPDF2

In [ ]:
tqdm.pandas()

def get_pdf_content(pdf_link):
    sleep(15)
    url = pdf_link
    # The default urllib user agent is blocked by the server
    r = urllib.request.Request(url, headers={'User-Agent': 'XYZ/3.0'})
    f = urllib.request.urlopen(r).read()
    f = BytesIO(f)
    pdfReader = PyPDF2.PdfFileReader(f)
    
    nb_pages = pdfReader.getNumPages()
    text = ''
    for i in range(0, nb_pages):
        pageObj = pdfReader.getPage(i)
        text = text + pageObj.extractText().replace('\n', '').strip()
    return text
    
df.loc[df['doctype'] == 'pdf', 'content'] = df.loc[df['doctype'] == 'pdf', 'link'].progress_apply(get_pdf_content)

In [ ]:
df.loc[[1771, 1778, 1786, 1787, 1788, 1790, 1808, 1815, 1819, 1823, 1830, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1880, 1891, 1892, 1893, 1894, 1895, 1913, 1914, 1921, 1924, 1929, 1937, 1940, 1946, 1951, 1952, 1953, 1956, 1957, 1965, 1973, 1987, 2005, 2007, 2008, 2028, 2029, 2030, 2031, 2061, 2072, 2075, 2079, 2087, 2090, 2101, 2111, 2112, 2113, 2114, 2115, 2122, 2135, 2137, 2138, 2139, 2140, 2145], 'content'] = None

In [ ]:
# For PDF files that PyPDF2 could not deal with
from pdfminer.high_level import extract_text

def get_hard_pdf_content(pdf_link):
    sleep(15)
    url = pdf_link
    r = urllib.request.Request(url, headers={'User-Agent': 'XYZ/3.0'})
    f = urllib.request.urlopen(r).read()
    f = BytesIO(f)
    text = extract_text(f).replace('\n', '').strip()
    return text

df.loc[(df['doctype'] == 'pdf') & (df['content'].isna()), 'content'] = df.loc[(df['doctype'] == 'pdf') & (df['content'].isna()), 'link'].progress_apply(get_hard_pdf_content)

## Step 4: cleaning content

In [1]:
import pandas as pd
df = pd.read_json('content.json', orient='table')

In [2]:
# Delete final paragraphs about the alignment of third countries with the EU's position
df['content'] = df['content'].str.replace(r'The Candidate Countries.*', '').str.strip()
# Delete introductions to conclusions
df.loc[df['content'].str.contains(r'\b1\. '), 'content'] = df.loc[df['content'].str.contains(r'\b1\. '), 'content'].str.extract(r'\b1\.(.*)', expand=False).str.strip()

# Delete item in French
df = df.drop([1790])
# Select appropriate content when scraping gave broader results
df.at[1080, 'content'] = "The European Council strongly condemns the continued assault on Aleppo by the Syrian regime and its allies, notably Russia and Iran, including the deliberate targeting of civilians and hospitals. The European Council urgently calls on the regime and Russia, as well as all parties in the Syrian conflict to implement immediately the following four emergency measures: a) the evacuation in safety and dignity of the inhabitants of the eastern part of Aleppo under the supervision and coordination of the United Nations, to a destination of their choosing. Members of the civil defence and civil administration must also be evacuated without obstruction under the supervision of the United Nations. The most seriously injured persons should be evacuated first; b) immediate and unconditional aid and protection for all inhabitants of the eastern part of Aleppo, without discrimination and in accordance with international humanitarian law, guaranteeing full and unimpeded access for the United Nations and its partners on the ground in supplying essential items and urgent medical assistance, as provided for in United Nations Security Council Resolution 2258, for the whole of Syria; c) genuine protection for all medical personnel and installations throughout the country, in accordance with United Nations Security Council Resolution 2286, and in particular for the border hospitals of Atmeh, Darkoush, Bab Al Hawa and Bab Al Salamah; d) international humanitarian law needs to be applied in the eastern part of Aleppo but also to the whole country and in particular to all those areas in which civilians have been besieged. The EU, as the first provider of humanitarian support to the Syrian population, will continue to work for achieving these goals. 27. Hostilities in Syria must cease immediately. The EU will work constructively with all partners, under UN auspices, towards a transition as agreed in United Nations Security Council Resolution 2254. To this end, the European Council invites the High Representative to continue her ongoing direct engagement with all relevant partners. Those responsible for breaches of international law, some of which may amount to war crimes, must be held accountable. The EU is considering all available options. The EU will provide support for Syria's reconstruction only once a credible political transition is firmly under way."
df.at[1771, 'content'] = "The European Council congratulates Ukraine on its new government and welcomes its determination to carry out political and economic reforms. Following the Commission´s second disbursement in December of 500 million Euro in macro-financial assistance, the EU and its Member States stand ready to further facilitate and support Ukraine's reform process, together with other donors and in line with IMF conditionality. The European Council welcomes the Commission's readiness to increase humanitarian aid to the suffering people in Ukraine. 6. The situation in eastern Ukraine remains a strong concern. The Union's policy of not recognising the illegal annexation of Crimea and Sevastopol was further tightened today. The EU will stay the course; the European Council is ready to take further steps if necessary. All parties, including Russia, should actively engage in and implement fully the Minsk agreements. The European Council calls for unhindered access to the crash site of MH17 in the interest of the ongoing investigations."
df.at[1778, 'content'] = "The EU reiterates its strong support for the UN's efforts to bring together key stakeholders in a dialogue on Libya's future. In this regard, it endorsed the efforts of the Special Representative of the UN Secretary General, Bernardino León, to take forward the Ghadames process. There is no alternative to a political solution based on dialogue, respect and trust. The EU condemns the violence which weighs heavily on the civilian population and undermines the prospects for a peaceful negotiated settlement. The recent military actions of the weekend are of particular concern. The EU calls for restraint on all sides and supports UN calls for a ceasefire so as to give the necessary space for inclusive national dialogue. The EU welcomes the engagement of those parties who have reacted positively to the UN initiative. The EU is concerned by the negative impact of the situation in Libya on other countries and calls on all those with an interest in the stability of Libya to support constructively the process.   Those who are undermining prospects for a political solution risk tipping Libya fully into civil war. They must face consequences for their actions. The EU's strong commitment to the unity and territorial integrity of Libya and the need to prevent the spread of terrorism, means that it remains ready to consider further actions, including restrictive measures, should circumstances so require.  The UN Security Council should also take measures against those obstructing dialogue."
df.at[1880, 'content'] = "Recalling the statements of the Heads of State or Government on Ukraine of 6 March and 27 May and its conclusions of 21 March, 27 June and 16 July, the European Council remains extremely concerned by the ongoing and increasingly intense fighting in Eastern Ukraine and continues to strongly condemn the illegal annexation of Crimea. It condemns the increasing inflows of fighters and weapons from the territory of the Russian Federation into Eastern Ukraine as well as the aggression by Russian armed forces on Ukrainian soil. It calls upon the Russian Federation to immediately withdraw all its military assets and forces from Ukraine. The European Union reiterates the urgent need for a sustainable political solution based on respect for Ukraine\'s sovereignty, territorial integrity, unity and independence. The European Council underlines the importance of implementing President Poroshenko\'s peace plan without delay. The first step should consist of a mutually agreed and viable cease-fire, the re-establishment of Ukrainian control over its border, and an immediate halt to the flow of arms, material and military personnel from the Russian Federation into Ukraine, as well as the urgent release of all hostages held by the illegally armed groups as well as of the prisoners detained in the Russian Federation. Moreover, the European Council reiterates its call for immediate, safe and unrestricted access to the MH17 crash site as part of a cease-fire. The European Council expresses once more its support for the valuable efforts of the Organisation for Security and Cooperation in Europe. 10.  The European Council welcomes the trilateral talks initiated between the European Union, Ukraine and the Russian Federation on practical issues with regard to the implementation of the Association Agreement/DCFTA and the talks on energy. The European Council calls on all parties to keep up the momentum in order to reach tangible results within the agreed timeframe. The European Council also calls on all parties to support and facilitate a stable and secure transport of energy sources, in particular gas. 11.  The European Council calls on all parties to support and facilitate without delay the work of international humanitarian organisations, in accordance with international humanitarian law and principles. The humanitarian impact of the conflict on the civilian population should not be exploited for political or military ends. The European Council calls on all contributors, including the Russian Federation, to support the international relief effort led by the United Nations, in full recognition of the Ukrainian Government\'s role as a first responder. 12.  The European Council remains engaged in the monitoring and assessment of the restrictive measures adopted by the European Union and stands ready to take significant further steps, in light of the evolution of the situation on the ground. It requests the Commission to urgently undertake preparatory work, jointly with the EEAS, and present proposals for consideration within a week. It requests the Commission to include in its proposal a provision on the basis of which every person and institution dealing with the separatist groups in the Donbass will be listed. 13.  The European Council welcomes the exceptional measures taken by the Commission to stabilise the EU agricultural and food markets in order to alleviate the effects of the Russian import restrictions on certain EU agricultural products. It invites the Commission to monitor the situation and to consider adopting any further measures, as appropriate."
df.at[1924, 'content'] = "Recalling the statements of the Heads of State or Government on Ukraine of 6 March and 27 May and its conclusions of 21 March and 27 June, the European Council once again stresses its support for a peaceful settlement of the crisis in Ukraine, notably the urgent need to agree on a genuine and sustainable cease-fire by all parties to create the necessary conditions for the implementation of President Poroshenko's peace plan. The European Council supports the diplomatic efforts by Ukraine, the Russian Federation, France and Germany, as well as the joint Berlin Declaration of 2 July. The European Council condemns the continuation of illegal activities by armed militants in Eastern Ukraine, including the occupation of public buildings, hostage-taking and armed attacks on Ukrainian law enforcers and border guards. The European Council urges the Russian Federation to actively use its influence over the illegally armed groups and to stop the flow of weapons and militants across the border, in order to achieve a rapid de-escalation. In this context, the European Council recalls the decision of 11 July to expand the travel ban and asset freeze within the European Union to 11 new individuals for actions undermining Ukraine's territorial integrity, sovereignty and independence. Member States also agreed to discontinue the application of their agreement of 20 February 2014 on export licences. 6. The European Council regrets that the requested steps it set out in its 27 June conclusions have not been adequately taken. As a result, the European Council agrees to expand the restrictive measures, with a view to targeting entities, including from the Russian Federation, that are materially or financially supporting actions undermining or threatening Ukraine's sovereignty, territorial integrity and independence. It tasks the Council to adopt the necessary legal instruments and to decide by the end of July on a first list of entities and persons, including from the Russian Federation, to be listed under the enhanced criteria. It also asks to consider the possibility of targeting individuals or entities who actively provide material or financial support to the Russian decision-makers responsible for the annexation of Crimea or the destabilisation of Eastern-Ukraine. The European Council requests the EIB to suspend the signature of new financing operations in the Russian Federation. European Union Member States will coordinate their positions within the EBRD Board of Directors with a view to also suspending financing of new operations. Finally, the European Council invites the Commission to re-assess EU-Russia cooperation programmes with a view to taking a decision, on a case by case basis, on the suspension of the implementation of EU bilateral and regional cooperation programmes. However, projects dealing exclusively with cross-border cooperation and civil society will be maintained. The European Council recalls that the Commission, the EEAS and the Member States have been undertaking preparatory work on targeted measures, as it requested in March, so that further steps can be taken without delay. The European Council remains committed to reconvene at any time should events so require. In line with the policy of non-recognition of the illegal annexation of Crimea and Sevastopol, the European Council requests the Commission and the EEAS to present proposals for additional measures in particular on restricting investments in Crimea and Sevastopol. The European Council also expects the International Financial Institutions to refrain from financing any projects that explicitly or implicitly recognise the illegal annexation of Crimea and Sevastopol. 7. The European Council commends the efforts of the OSCE and its Chairmanship in Office, particularly in facilitating meetings of the Contact Group in Ukraine, and its readiness to establish a border-monitoring mission, to which the European Union and its Member States stand ready to consider a substantial contribution. 8. The European Council stresses the European Union's commitment to pursue trilateral talks on the conditions of gas supply from the Russian Federation to Ukraine and commends the efforts of the Commission in that regard. Finding a swift agreement is important for safeguarding the security of supply and transit of natural gas through Ukraine to EU Member States and for the stabilisation of Ukraine's economy. The European Council stresses the importance of Ukraine ratifying the Association Agreement with a view to its early provisional application. It welcomes the holding of trilateral consultations at ministerial level between Ukraine, the Russian Federation and the European Union on 11 July on the implementation of the Association Agreement. In this context, it also welcomes the setting up of a consultation mechanism to address potential difficulties resulting from the effects of the implementation of the Deep and Comprehensive Free Trade Area on the implementation of the Free Trade Agreement of the Commonwealth of Independent States."
df.at[1940, 'content'] = "Recalling the Statements of the Heads of State and Government on Ukraine of 6 March and 27 May, the European Council conclusions of 21 March and the conclusions of the Foreign Affairs Council on Ukraine of 23 June, the European Council expresses its support to the peace plan announced last week by President Poroshenko. It takes note of the Russian President\'s declaration of support in principle of the peace plan and the Federation Council\'s decision to revoke the authorisation to use Russian forces to intervene militarily in Ukraine. 30.  The European Council regrets that the cease-fire, while being respected by the Ukrainian authorities, has not led to the full cessation of military hostilities. Therefore, it calls upon all parties to genuinely commit to the implementation of the peace plan and to cement the cessation of the military activities. It urges the Russian Federation to actively use its influence over the illegally armed groups and to stop the flow of weapons and militants across the border, in order to achieve rapid and tangible results in de-escalation. The European Council supports the OSCE monitoring the implementation of the peace plan as well as its role in supporting the cease-fire and the establishment of effective border controls. 31.  The European Council reconfirms its commitment to support the economic stabilisation process in Ukraine and welcomes the two recent significant Commission disbursements totalling 750 million EUR in the framework of the State Building Contract and the Macro Financial Assistance. In this context, the European Council looks forward to the high level donor coordination meeting on Ukraine to be held in Brussels on 8 July. 32.  Following its March conclusions and the decision not to recognise the illegal annexation of Conclusions – 26/27 June 2014 Crimea and Sevastopol, the European Council welcomes the work undertaken by the Commission to give effect to this policy and the decision to prohibit the import of goods from Crimea and Sevastopol which do not have a Ukrainian certificate. 33.  The European Council recalls that the European Commission, the EEAS and the Member States have been undertaking preparatory work on targeted measures, as it requested in March, so that further steps can be taken without delay. In that respect, the European Council expects that by Monday 30 June the following steps will have been taken : agreement on a verification mechanism, monitored by the OSCE, for the cease-fire and for the effective control of the border; return to the Ukrainian authorities of the three border checkpoints (Izvarino, Dolzhanskiy, Krasnopartizansk); release of hostages including all of the OSCE observers; launch of substantial negotiations on the implementation of President Poroshenko\'s peace plan. The Council will assess the situation and, should it be required, adopt necessary decisions. The European Council underlines its commitment to reconvene at any time for further significant restrictive measures."
df.at[826, 'content'] = "The European Union and its Member States regret the visit of President Omar Al-Bashir to Uganda, a State party to the Rome Statute of the International Criminal Court (ICC). The EU calls upon all Member States of the United Nations to abide by and implement the resolutions adopted by the Security Council under Chapter VII of the UN Charter, notably UNSCR 1593(2005). The EU urges Uganda to respect its obligations under international law and as a State Party to the ICC. The European Union is committed to enforcing international criminal law and to ending impunity and remains a staunch supporter of the ICC."

df['content'] = df['content'].str.replace("\\", '')
df['content'] = df['content'].str.replace('\(cid:5\)', 'N')
df['content'] = df['content'].str.replace('\(cid:6\)', 'N')

## Step 5: finding the most frequent words

In [3]:
pd.Series(' '.join(df['content']).lower().split()).value_counts()[:50]

the              18231
and               8465
of                7572
to                6631
in                4691
eu                2519
a                 2123
on                1977
for               1905
its               1575
with              1487
council           1302
as                1249
by                1230
all               1121
that              1120
is                1011
be                 852
international      818
this               796
support            794
it                 765
will               758
european           729
political          721
including          686
an                 583
security           566
humanitarian       552
calls              524
human              522
which              513
their              493
union              485
rights             468
are                449
efforts            420
at                 411
also               411
government         389
welcomes           387
has                351
un                 343
through    

A raw search returns a lot of articles, particles and conjunctions that are not meaningful, we need to refine it. One approach is to identify expressions that follow our subject, referred as "the EU", "the European Union", "the European Council", "the Council", or simply "we".

In [4]:
import re
expressions = df['content'].str.findall(r'the council\s+((?:[\w\']+(?:\s+|$)){5})|the european council\s+((?:[\w\']+(?:\s+|$)){5})|the eu\s+((?:[\w\']+(?:\s+|$)){5})|the european union\s+((?:[\w\']+(?:\s+|$)){5})|\bwe\s+((?:[\w\']+(?:\s+|$)){5})', flags=re.IGNORECASE).explode()

In [5]:
# Remove empty tuples
def rem_empty_tuples(tup):
    tup = [t for t in tup if t]
    return tup

expressions = expressions.apply(rem_empty_tuples)

# Convert to strings
expressions = expressions.apply(''.join)

In [6]:
expressions.value_counts()[:50]

calls on all parties to                      22
Rue de la Loi 175                            21
and its Member States will                   15
and its Member States are                    15
is deeply concerned by the                   10
stands ready to support the                   9
reiterates its support for the                8
will continue to support the                  8
invites the High Representative to            8
reiterates its call for the                   7
confirms the EU's readiness to                7
reiterates its firm commitment to             7
and its Member States call                    7
calls on all parties in                       7
reiterates its full support for               7
and its Member States have                    6
condemns in the strongest terms               6
and its Member States stand                   6
calls on all sides to                         6
reiterates its full support to                6
commends the efforts of the             

We find indeed that "is deeply concerned by" is one of most frequent expressions used in EU foreign policy statements. We can also search other similar expressions that contain the word "concern".

In [7]:
print('Number of expressions containing the word "concern" or "concerned":', expressions[expressions.str.contains('concern')].shape[0], 'in a total of', expressions.shape[0])

Number of expressions containing the word "concern" or "concerned": 184 in a total of 3305


In [8]:
concern_levels = expressions.str.extract(r'(\w+)\sconcern', flags=re.IGNORECASE, expand=False).dropna()
concern_levels.value_counts()

deeply          37
deep            18
is              17
its             12
remains         12
extremely       10
grave            9
seriously        8
serious          8
the              5
expresses        5
particularly     5
very             4
also             4
gravely          3
be               3
increasingly     3
growing          3
are              2
parties          2
utmost           2
with             2
profound         2
ongoing          1
deepest          1
all              1
likewise         1
of               1
strong           1
great            1
highest          1
Name: content, dtype: int64

Deep and deeply are the most frequent level of concern, but the whole spectrum is quite wide: ongoing concern, growing concern, increasingly concerned, very concerned, particularly concerned, serious concern, great concern, profound concern, strong concern, grave concern, extreme concern, highest concern, deepest concern, utmost concern.

In [9]:
pd.Series(' '.join(expressions).lower().split()).value_counts()[:50]

the           1850
to             980
of             593
its            548
and            350
on             329
that           307
calls          293
welcomes       283
is             242
will           239
reiterates     227
all            221
for            210
support        188
also           169
in             149
by             148
member         135
states         132
continue       128
a              113
concerned      112
commitment     110
importance     105
recalls        102
remains        100
parties        100
condemns        98
ready           97
urges           94
supports        91
with            89
efforts         89
underlines      85
strongly        82
call            80
has             78
encourages      75
committed       74
reaffirms       74
government      74
expresses       69
stands          64
concern         61
about           60
commends        60
need            59
work            59
fully           58
dtype: int64

When narrowing our search of the most frequent words only in expressions, we find that other common EU actions are calls, welcomes, repetitions of previously expressed positions (reiterate, continue, recall, remain), but also supports, commitments, encouragements or condemnations and urges.

A special mention should be made about a declaration from February 2018 by the High Representative on behalf of the EU on Syria, about which "the European Union **is running out of words** to describe the horror being experienced by the people of Eastern Ghouta". Sometimes a lack of words can be the strongest expression...

In [10]:
df.loc[752, 'content']

'The European Union is running out of words to describe the horror being experienced by the people of Eastern Ghouta. Hundreds of civilians, women, children are targeted deliberately and relentlessly. They are the real, innocent, victims of this war. Unhindered humanitarian access and the protection of civilians is a moral duty and a matter of urgency. It is in the responsibility of all to prevent further loss of lives, to stop the violence. The fighting must stop now - the international community should unite to stop this human suffering. The European Union calls on all parties to the conflict, as well as the guarantors of the four De-Escalation Areas, to take all necessary measures to ensure an immediate ceasefire, the protection of the Syrian people by respecting International Humanitarian Law, and urgent humanitarian access. The Syrian regime must immediately stop targeting its own people and fulfil its primary responsibility to protect them. We therefore support the UN call for a 

## Step 6: identifying the most mentioned countries and associated expressions 

In [11]:
top_10_countries = df['main_topic'].value_counts()[:10].to_dict()
top_10_countries

{'Ukraine': 29,
 'Venezuela': 26,
 'Syria': 25,
 'Libya': 20,
 'South Sudan': 13,
 'Sudan': 12,
 'Middle East': 12,
 'Afghanistan': 11,
 'Yemen': 9,
 'Bosnia and Herzegovina': 9}

In [12]:
for country in top_10_countries:
    print(country)
    print(expressions.loc[df[df['main_topic'] == country].index].value_counts()[:5])
    print('\n')

Ukraine
calls on all parties to                     7
calls again on UN Member                    5
remains committed to fully implementing     4
reiterates its full support for             4
calls for full compliance with              4
Name: content, dtype: int64


Venezuela
will continue to monitor the              3
is ready to start work                    2
considers that the voting session         2
reiterates its readiness to cooperate     2
calls on the Venezuelan Government        2
Name: content, dtype: int64


Syria
will continue to support the                   6
and its Member States will                     3
calls upon all parties to                      3
reiterates that there can be                   3
imposed additional restrictive measures on     3
Name: content, dtype: int64


Libya
calls on all parties in                   5
remains strongly committed to the         4
and its Member States call                3
welcomes the UN humanitarian response     2
emphasize